# Landfall decay rates for tropical cyclones in the Australian region

This notebook analyses the historical best track data from [IBTrACS](http://www.ncdc.noaa.gov/oa/ibtracs/) to evaluate the rate of decay of cyclones after landfall. 

In previous studies (e.g. Vickery, 2005), the rate of change of central pressure deficit ($\Delta p_c$) after landfall is modelled as an exponential decay function:

$\Delta p_c(t) = \Delta p_0 e^{(-\alpha t)}$

where $\alpha = \alpha_0 + \alpha_1 \Delta p_0$ and $\Delta p_0$ is the central pressure deficit at landfall. The decay rate $\alpha$ may also be a function of the translation speed after landfall ($v_t$) and/or the landfall latitude. Here, we are attempting to fit this model to TC occurrences in the Australian region. We begin with the simplest model - where $\alpha$ is a function of $\Delta p_0$ alone. 

In [1]:
%matplotlib inline

from __future__ import print_function

import sys
import numpy as np
import pandas as pd

from scipy.optimize import leastsq
import scipy.stats as stats
from scipy.stats import logistic as slogistic
from scipy.stats import norm as snorm
import io

from matplotlib import pyplot as plt
from matplotlib.cm import get_cmap
from mpl_toolkits.basemap import Basemap

import statsmodels.api as sm

from Utilities.loadData import loadTrackFile
from Utilities.config import ConfigParser
from Utilities.track import Track
from Utilities.metutils import convert

from Evaluate.interpolateTracks import parseTracks

import seaborn
seaborn.set_context("poster")
seaborn.set_style("whitegrid")

Define the configuration settings for the analysis. You will need to have the latest version of the IBTrACS data and the daily long term mean sea level pressure data stored locally. Modify the path to those files as required.

To modify the region analysed, change the `gridLimit` option in the `Region` section. You can restrict the TC seasons used by modifying the `StartSeason` option in the `DataProcess` section. 

In [27]:
configstr = """
[DataProcess]
InputFile=C:/WorkSpace/data/TC/Allstorms.ibtracs_wmo.v03r06.csv
Source=IBTRACS
StartSeason=1981
FilterSeasons=True

[Region]
; Domain for windfield and hazard calculation
gridLimit={'xMin':110.,'xMax':155.,'yMin':-30.0,'yMax':-5.0}
;gridLimit={'xMin':250.,'xMax':340.,'yMin':10.0,'yMax':40.0}
gridSpace={'x':1.0,'y':1.0}
gridInc={'x':1.0,'y':0.5}

[Input]
landmask = C:/WorkSpace/tcrm/input/landmask.nc
mslpfile = C:/WorkSpace/data/MSLP/slp.day.ltm.nc
datasets = IBTRACS,LTMSLP

[IBTRACS]
; Input data file settings
url = ftp://eclipse.ncdc.noaa.gov/pub/ibtracs/v03r06/wmo/csv/Allstorms.ibtracs_wmo.v03r06.csv.gz
path = C:/WorkSpace/data/TC/
filename = Allstorms.ibtracs_wmo.v03r06.csv
columns = tcserialno,season,num,skip,skip,skip,date,skip,lat,lon,skip,pressure
fielddelimiter = ,
numberofheadinglines = 3
pressureunits = hPa
lengthunits = km
dateformat = %Y-%m-%d %H:%M:%S
speedunits = kph

[LTMSLP]
; MSLP climatology file settings
URL = ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/surface/slp.day.1981-2010.ltm.nc
path = C:/WorkSpace/data/MSLP
filename = slp.day.ltm.nc
"""

Load the configuration settings from the string representation above. Normally, this would be stored in a file and read directly from there. 

In [28]:
config = ConfigParser()
config.readfp(io.BytesIO(configstr))

Load the track file. This reports the number of tracks loaded - this varies based on the `StartSeason` option in the `DataProcess` section above. Tracks for all basins are loaded by default, and then we filter spatially at a later step.

In [29]:
trackFile = config.get('DataProcess', 'InputFile')
source = config.get('DataProcess', 'Source')

print("Track file: {0}".format(trackFile))
print("Track format: {0}".format(source))
tracks = loadTrackFile(configstr, trackFile, source)
#tracks = parseTracks(configstr, trackFile, source, 1.)
print("There are {0:d} tracks in the input dataset".format(len(tracks)))

Set up a map to define the study region. I have to set up different keyword arguments for `Basemap` for plotting using `saveTrackMap`, because it cannot use a cylindrical projection (it baulks at setting up a scale bar). And we can't use a Mercator projection in the map used for testing if TCs are on land or not. 

In [30]:
domain = config.geteval('Region', 'gridLimit')
mapkwargs = dict(llcrnrlon=domain['xMin'],
                 llcrnrlat=domain['yMin'],
                 urcrnrlon=domain['xMax'],
                 urcrnrlat=domain['yMax'],
                 resolution='c',
                 projection='cyl')
m = Basemap(**mapkwargs)


This function identifies the landfall pressure deficit and time, then extracts the pressure deficit at all times after landfall.

It returns the initial pressure deficit $\Delta p_{0}$, the pressure deficit $\Delta p_c (t)$ at each subsequent point in time, and the time after landfall $t$ (in hours). Only those storms where there are at least two observations _after landfall_ are returned. 

Presently, the function returns once a valid storm is found and either ends or moves offshore. This misses the second or later landfalls of those storms that make multiple landfalls. 

In [31]:
def processTrack(track, m):
    onland = np.zeros(len(track.Longitude))
    dp0 = 0
    v0 = 0
    for i, (lon, lat) in enumerate(zip(track.Longitude, 
                                      track.Latitude)):
        if m.is_land(lon, lat):
            onland[i] = 1

    dp = []
    dt = []
    v = []
    flag = 0
    for i in range(1, len(onland)):
        if (onland[i]==1) & (onland[i-1]==0) & (track.CentralPressure[i-1] > 0.0):
            # New landfall (with central pressure prior to landfall):
            t0 = track.TimeElapsed[i-1]
            dp0 = track.EnvPressure[i-1] - track.CentralPressure[i-1]
            v0 = track.Speed[i-1]
            lat0 = track.Latitude[i-1]
            flag = 1
        
        if (flag==1) & (track.CentralPressure[i] > 0.0):
            # Storm is on land and has a valid central pressure record:
            dp.append(track.EnvPressure[i] - track.CentralPressure[i])
            dt.append(track.TimeElapsed[i] - t0)
            v.append(track.Speed[i])
            flag = onland[i]

            if flag == 0:
                return dp0, dp, dt, v0, np.mean(np.array(v)), lat0
    if len(dp) > 1:
        return dp0, dp, dt, v0, np.mean(np.array(v)), lat0
    else:
        return None, None, None, None, None, None
        

Define a pair of functions to fit the pressure deficit (as a function of time after landfall) to an exponential decay model. The `minimise` function returns the $\alpha$ parameter for an event.

Cycle through all the tracks in the input dataset. If the track passes through the study region, then we will process it to obtain the decay rate as a function of $\Delta p_0$ and $t$. The fitted decay rate is added to a list of values, as is the landfall pressure value. 

The time history of each event is then plotted -- both the central pressure deficit ($\Delta p_c(t)$) and the central pressure deficit normalised by the landfall pressure deficit ($\Delta p_0$).

In [32]:
def residuals(params, dp, dt, dp0):
    yfit = dp0 * np.exp(-(np.array(params) * dt))
    return dp - yfit

def minimise(dp, dt, dp0, alpha=0., beta=1.):
    plsq = leastsq(residuals, [alpha], args=(dp, dt, dp0))
    return plsq[0]

In [33]:
def plotEvent(ax, dt, dp, dp0, params):
    ax.scatter(dt, dp/dp0, marker='s')
    xt = ax.get_xticks()
    ax.set_xlabel("Time after landfall (hours)")
    ax.set_xticks(range(0,12*int(1+max(xt)/12.+1),12))
    ax.set_ylabel(r"$\frac{\Delta p_c(t)}{\Delta p_0}$ (hPa)")
    xm = np.linspace(0, 12*int(max(dt)/12.), 100)    
    ax.set_xlim((0,12*int(1+max(xt)/12.)))

    legtext = r"$\Delta p_c = \Delta p_0 \exp{(%f t)}$"%(-params[0])
    ym = np.exp(-params[0]*xm)
    ax.plot(xm, ym, label=legtext)
    (ymin, ymax) = ax.get_ylim()
    l = ax.legend(loc=0)
    ax.set_ylim((0,ymax))

If you want to plot individual tracks and the fitted decay model, uncomment the lines in the innermost if-clause. 

In [56]:
landfall_pressure = []
landfall_speed = []
landfall_lat = []
decayrate = []
lftracks = []
nevents=0
fig1, (ax1, ax2) = plt.subplots(2,1,sharex=True)

for n, track in enumerate(tracks):
    if track.inRegion(domain):
        # Process the track to get the pressure at landfall and the decay rate thereafter:
        dp0, dp, dt, v0, v, lat0 = processTrack(track, m)
        if (dp0 is not None)  and (min(dp) > 0) and (dp0 >= max(dp)):
            nevents += 1
            lftracks.append(track)
            p = minimise(dp, dt, dp0, 0., 1.)
            ax1.plot(dt, dp)
            ax2.plot(dt, (dp/dp0))
            #fig0, ax = plt.subplots(1,1,sharex=True)
            #plotEvent(ax, dt, dp, dp0, p)
            #ax.set_title("Storm {0} ({1}) ".format(n, track.Year[0]))
            #plt.savefig("{0:03d}.png".format(n))

            decayrate.append(p[0])
            landfall_pressure.append(dp0)
            landfall_speed.append(v0)
            landfall_lat.append(lat0)
    
ax2.set_xlabel("Time after landfall (hours)")
xt = ax2.get_xticks()
ax2.set_xticks(range(0,12*int(1+max(xt)/12.),12))
ax1.set_ylabel("$\Delta p_c(t) $ (hPa)")
ax1.set_title("Pressure deficit decay rates ({0} events)".format(nevents))
ax2.set_ylabel(r"$\frac{\Delta p_c(t)}{\Delta p_0}$")


The events include all storms that make landfall in Australia - including those where $\Delta p$ increases *after* landfall. As we will see, this generally occurs with weaker TCs (smaller $\Delta p_0$). 

### Linear regression model

We use a linear regression to fit the decay rate $\hat{\alpha}$ as a model of landfall pressure deficit $\Delta p_0$. The `seaborn.jointplot()` command fits a linear regression to the data, and adds the 95% approximate confidence interval (shaded), based on bootstrap resampling (1000 samples). 

In [57]:
df = pd.DataFrame({'alpha':decayrate,
                   'dp0':landfall_pressure,
                   'v0':landfall_speed,
                   'lat0':landfall_lat})
jp = seaborn.jointplot('dp0','alpha',df, kind='reg', size=10,xlim=(0,140))
jp.set_axis_labels(r"${\Delta p_0}$", r"$\hat{\alpha}$")

We use the [`statmodels`](http://statsmodels.sourceforge.net/) package to fit our model of landfall decay rate, as a function of landfalling pressure deficit. The first model to fit is a simple linear model using ordinary least squares:

$\hat{\alpha} = \alpha_0 + \alpha_1 \Delta p_0 + \varepsilon $

This includes a random variate ($\varepsilon$) to allow for variations from the fitted decay rate in the modelled values.

In [58]:
X = sm.add_constant(landfall_pressure)
y = np.array(decayrate)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())
print(results.params)
print(results.rsquared)
print(stats.mstats.normaltest(results.resid))
print(stats.kstest(results.resid, 'norm'))

Based solely on landfall pressure deficit $\Delta p_0$ from 113 events, the decay rate can be modelled as:

$\hat{\alpha} = -0.001479 + 0.001061 \Delta p_0 + \varepsilon$.

#### Model residuals

The residuals of the regression model can be examined to provide further random variations to the stochastic model of landfall decay rates. The figure below shows the model residuals $\varepsilon$ against $\Delta p_0$. 

In [59]:
rp = seaborn.residplot('dp0','alpha',df,lowess=False)
rp.set_xlabel(r"${\Delta p_0}$")
rp.set_xlim((0,120))
rp.set_ylabel(r"$\varepsilon$")
rp.set_title(r"Model residuals: $\sigma^2 = ${0:.4f}".format(np.std(results.resid)))

A non-central Student's T (NCT) distribution is fitted to the residuals (below, left). The NCT distribution provides a narrower distribution than the normal distribution, leading to a better fit to the observed residuals. The quantile-quantile plot (below, right, with scaled quantiles) provides a clear indication of the fit to the model residuals. 

In [60]:
fig, (ax0, ax1) = plt.subplots(1,2, figsize=(14,6))

bins = np.arange(-0.1, 0.21, 0.01)
ax = seaborn.distplot(results.resid,bins=bins, ax=ax0, 
                      kde_kws={'label':'Residuals','linestyle':'--'}, 
                      norm_hist=True)

fp = stats.nct.fit(results.resid)
print("Fit parameters for the non-central Student's T distribution:")
print(fp)

x = np.linspace(-0.1, 0.2, 1000)



ax.set_ylabel("Count")
ax.set_xlabel(r"$\varepsilon$")
pp = sm.ProbPlot(results.resid, stats.nct, fit=True)
pp.qqplot('Non-central T', 'Residuals', line='45', ax=ax1, color='gray',alpha=0.5)
fig.tight_layout()
print(pp.fit_params)
ppfit = pp.fit_params

ax.plot(x, stats.nct.pdf(x,fp[0], fp[1], loc=fp[2], scale=fp[3]), label='Non-central T')
ax.legend(loc=0)

In [61]:
fig, (ax0, ax1) = plt.subplots(1,2, figsize=(14,6))

bins = np.arange(-0.1, 0.21, 0.01)
ax = seaborn.distplot(results.resid, bins=bins, ax=ax0, kde_kws={'label':'Residuals','linestyle':'--'})
n, b = np.histogram(results.resid, bins=bins, density=True)
fpnorm = stats.norm.fit(results.resid)#, floc=np.median(results.resid),scale=np.std(results.resid))
print("Fit parameters for the normal distribution:")
print(fpnorm)

x = np.linspace(-0.1, 0.2, 1000)
ax.plot(x, stats.norm.pdf(x, *fpnorm),label='Normal')
ax.legend(loc=0)


ax.set_ylabel("Count")
ax.set_xlabel(r"$\varepsilon$")
ppnorm = sm.ProbPlot(results.resid, stats.norm, fit=True)
ppnorm.qqplot('Normal', 'Residuals', line='45', ax=ax1, color='gray',alpha=0.5)
fig.tight_layout()


In [62]:
fig, (ax0, ax1) = plt.subplots(1,2, figsize=(14,6))

bins = np.arange(-0.1, 0.21, 0.01)
ax = seaborn.distplot(results.resid, bins=bins, ax=ax0, kde_kws={'label':'Residuals','linestyle':'--'})
n, b = np.histogram(results.resid, bins=bins, density=True)
fplog = stats.logistic.fit(results.resid)#, loc=np.median(results.resid), scale=np.std(results.resid))
print("Fit parameters for the logistic distribution:")
print(fplog)

x = np.linspace(-0.1, 0.2, 1000)
ax.plot(x, stats.logistic.pdf(x, *fplog),label='Logistic')
ax.legend(loc=0)


ax.set_ylabel("Count")
ax.set_xlabel(r"$\varepsilon$")
pplog = sm.ProbPlot(results.resid, stats.logistic, fit=True)
pplog.qqplot('Logistic', 'Residuals', line='45', ax=ax1, color='gray',alpha=0.5)
fig.tight_layout()

Here, we perform a Kolmogorov-Smirnov test of the residuals against a random sample from a base distribution. We test against the non-central T, normal and logistic distributions, each of which is fitted to the residuals. The test statistic for the non-central T distribution is lowest, justifying our choice of that distribution for random innovations.

In [63]:
print(stats.kstest(results.resid, 'nct', args=pp.fit_params))
print(stats.kstest(results.resid, 'norm', args=ppnorm.fit_params))
print(stats.kstest(results.resid, 'logistic', args=pplog.fit_params))

#### Results

The model chosen for the TC landfall decay model is as follows:

$\hat{\alpha} = -0.001479 + 0.001061 \Delta p_0 + \varepsilon(\nu, \mu, \lambda, \sigma)$

$\varepsilon(\nu, \mu, \lambda, \sigma)$ is a random variate from a non-central Student's T distribution, with $\nu = 12.283$ degrees of freedom, non-centrality parameter $\mu = 8.559$, location parameter $\lambda = -0.108$ and scale parameter $\sigma = 0.0118$.

Here we demonstrate application of the landfall decay model, incorporating random innovations that reflect the variance explained by the linear regression model. Random innovations are sampled from an NCT with the degrees or freedom, noncentrality, location and scale given by the fitted results above. The modelled $\alpha$ values show wide scatter around the linear model, comparable to the scatter in the observed data (above). Notably, the model does not generate negative values for $\alpha$, matching the observations (which were selected to have $\alpha > 0$).

In [64]:
r = (results.rsquared)
pr = results.params
p0 = np.random.choice(landfall_pressure, 178, replace=True)

pp = (pr[0] + pr[1]*p0) + stats.nct.rvs(fp[0], fp[1], loc=fp[2],
                                        scale=fp[3], size=178)

fig, ax1 = plt.subplots(1, 1)

seaborn.regplot('dp0', 'alpha', df, scatter_kws={'s':25}, label="Observed", ax=ax1)
ax1.set_xlim((0, 100))
ax1.set_ylim((-0.05, 0.20))
ax1.set_xlabel(r"${\Delta p_0}$ (hPa)")
ax1.set_ylabel(r"$\hat{\alpha}$")
ax1.axhline(0.0, color='k')

ax1.scatter(p0, pp, s=25, color='red', marker='s', 
            alpha=0.5, label="Modelled")
ax1.legend(loc=0)


Notice that this model does permit a negative rate parameter (see the 95th percentile value for the intercept). i.e. it's possible that the central pressure will increase with time after landfall. This is feasible, since there are many observed TCs that record increases in intensity after landfall. However, for more intense TCs, this becomes less likely. 

### Other parameters

But what about the translation speed at landfall? Vickery (2005) suggests that translation speed may influence the decay rate. So we plot the average translation speed at landfall ($v_t$) against $\Delta p_0$, with contours of $\alpha$. 

In [65]:
seaborn.interactplot("dp0", 'v0', 'alpha', df, cmap=get_cmap("YlOrRd"))

So it appears that there may be a relationship between $\alpha$ and $v_t$. To confirm this, we put together a linear model of $\alpha = f(\Delta p_0, v_t)$. The resulting fit indicates an improvement in the $R^2$ value, and the AIC score is very slightly decreased.

In [66]:
X = np.column_stack((landfall_pressure, landfall_speed))
X = sm.add_constant(X)
y = np.array(decayrate)
var2model = sm.OLS(y, X)
var2results = var2model.fit()
print(var2results.summary())
print('Parameters: ', var2results.params)
print('R-squared: ', var2results.rsquared)
print('P-value: ', var2results.pvalues)

In [67]:
fp2d = stats.nct.fit(var2results.resid, loc=np.mean(var2results.resid),
                     scale=np.std(var2results.resid))
pr = var2results.params
p0 = np.random.choice(landfall_pressure, 55, replace=True)
v0 = np.random.choice(landfall_speed, 55, replace=True)

rpoints = pr[0] + pr[1]*p0 + pr[2]*v0 + \
            stats.nct.rvs(fp2d[0], fp2d[1], 
                          loc=fp2d[2], scale=fp2d[3],
                          size=55)

dp = np.arange(0, 101, 1)
vv = np.arange(0, 40.1, 0.5)

xx, yy = np.meshgrid(dp, vv)
pp = pr[0] + pr[1]*xx + pr[2]*yy
fig, ax1 = plt.subplots(1, 1)
CS = ax1.contourf(xx, yy, pp, 10,extend='both')
plt.colorbar(CS, extend='both', label='Decay rate (hPa/hr)')
ax1.set_xlim((0, 100))
ax1.set_ylim((0, 20))
ax1.set_xlabel(r"${\Delta p_0}$ (hPa)")
ax1.set_ylabel(r"$v_0$ ($\mathrm{m} \mathrm{s}^{-1}$)")
pal1 = seaborn.light_palette("Red", as_cmap=True)
pal2 = seaborn.light_palette("Blue", as_cmap=True)
ax1.scatter(landfall_pressure, landfall_speed, s=50, marker='s', 
            c=decayrate, cmap=pal1, label="Observed")
ax1.scatter(p0, v0, s=50, marker='o', c=rpoints, cmap=pal2,  
            label="Modelled")
ax1.legend(loc=0, frameon=True)
print(fp2d)
print(rpoints.min(), rpoints.max())

In [68]:
fig1, (ax0, ax1) = plt.subplots(1,2, figsize=(14,6))
bins = np.arange(-0.1, 0.21, 0.01)
ax = seaborn.distplot(var2results.resid, bins=bins, 
                      ax=ax0, kde_kws={'label':'Residuals','linestyle':'--'})
#n, b = np.histogram(var2results.resid, bins=bins, density=False)
fp2d = stats.nct.fit(var2results.resid)#, 
                     #loc=np.median(var2results.resid), 
                     #scale=np.std(var2results.resid))
print("Fit parameters for the non-central Student's T distribution:")
print(fp2d)
x = np.linspace(-0.1, 0.2, 1000)

ax.plot(x, stats.nct.pdf(x, fp2d[0], fp2d[1], 
                         loc=fp2d[2], scale=fp2d[3]), 
        label='Non-central T')
ax.legend(loc=0)


ax.set_ylabel("Count")
ax.set_xlabel(r"$\varepsilon$")
ax.set_ylim(0,25)
ax.set_xlim(-0.1, 0.2)
var2pp = sm.ProbPlot(var2results.resid, stats.nct, fit=True)
var2pp.qqplot('Non-central T', 'Residuals', 
              line='45', color='gray', alpha=0.5, ax=ax1)
print(var2pp.fit_params)

The 2-parameter model for the TC landfall decay model is as follows:

$\alpha = 0.0115 + 0.00022 \Delta p_0 + 0.0015 v_0 + \varepsilon(\nu, \mu, \lambda, \sigma)$

$\varepsilon(\nu, \mu, \lambda, \sigma)$ is a random variate from a non-central Student's T distribution, with $\nu = 12.57$ degrees of freedom, non-centrality parameter $\mu = 9.215$, location parameter $\lambda = -0.1097$ and scale parameter $\sigma = 0.0112$.

In [69]:
var2ppnorm = sm.ProbPlot(var2results.resid, stats.norm, fit=True)
var2pplog = sm.ProbPlot(var2results.resid, stats.logistic, fit=True)


Here, we perform a Kolmogorov-Smirnov test of the residuals against a random sample from a base distribution. We test against the non-central T, normal and logistic distributions, each of which is fitted to the residuals. The test statistic for the non-central T distribution is lowest, justifying our choice of that distribution for random innovations.

In [70]:
print(stats.kstest(var2results.resid, 'nct', args=var2pp.fit_params))
print(stats.kstest(var2results.resid, 'norm', args=var2ppnorm.fit_params))
print(stats.kstest(var2results.resid, 'logistic', args=var2pplog.fit_params))


Latitude at landfall has a lesser influence on decay rate. The resulting model has a higher AIC value compared to the simple model where $\alpha$ is a function of landfall pressure deficit only.

In [71]:
ax = seaborn.interactplot("dp0", 'lat0', 'alpha', df, cmap=get_cmap("YlOrRd"),)
ax.set_xlabel(r'$\Delta p_0$')
ax.set_ylabel('Landfall latitude')


In [53]:
X = np.column_stack((landfall_pressure, landfall_lat))
X = sm.add_constant(X)
y = np.array(decayrate)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())
print('Parameters: ', results.params)
print('P-value: ', results.pvalues)
print('R-squared: ', results.rsquared)

#### Contributing events

Finally, we plot the tracks of all landfalling TCs used in the analysis. This uses some code written for [TCRM](https://github.com/GeoscienceAustralia/tcrm), so you'll need to have the code somewhere on the `PYTHONPATH` to get this to work. Notice also this is actually saving the image to a file, then displaying that image file, as opposed to the previous plots which are displayed inline. 

In [100]:
from PlotInterface.tracks import saveTrackMap
from IPython.display import Image

In [171]:
startSeason = config.get("DataProcess", "StartSeason")
xx = np.arange(domain['xMin'], domain['xMax'] + 0.1, 0.1)
yy = np.arange(domain['yMin'], domain['yMax'] + 0.1, 0.1)

[xgrid, ygrid] = np.meshgrid(xx,yy)
title = "Landfalling TCs - {0} - 2013".format(startSeason)
mapkwargs = dict(llcrnrlon=domain['xMin']-10,
                 llcrnrlat=domain['yMin'],
                 urcrnrlon=domain['xMax'],
                 urcrnrlat=domain['yMax'],
                 resolution='f',
                 projection='merc')

saveTrackMap(lftracks, xgrid, ygrid, title, mapkwargs, "tracks.png")
Image("tracks.png")

This document was written in an IPython notebook. The raw notebook can be downloaded [here](https://github.com/wcarthur/notebooks/blob/master/TC%20landfall%20decay.ipynb). See also [nbviewer](http://nbviewer.ipython.org/github/wcarthur/notebooks/blob/master/TC%20landfall%20decay.ipynb) for an online static view.